# Setup

In [1]:
import torch
import numpy as np
import json
import torch.nn as nn
from birdlib import utils

In [2]:
# sudo modprobe nvidia_uvm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [39]:
DATASET_NAME = "dataset"
MODEL_NAME = 'DeeperCNN'
DATASET_VAR = 'base'

In [40]:
DATASET_PATH = f'../segments/{DATASET_NAME}'
TRAIN_PATH = f"{DATASET_PATH}/train"
TEST_PATH = f"{DATASET_PATH}/test"
MODEL_PATH = f'./models/{MODEL_NAME}'

# Load the model

In [41]:
with open(f"./utils/{DATASET_NAME}/dataset_config_{DATASET_VAR}.json") as f:
    dataset_config = json.load(f)

In [42]:
mappings = dataset_config["mappings"]

In [43]:
model_class = utils.load_model_class(MODEL_NAME)
model = model_class(len(mappings))
model.to(device)
saving_path = f'models/{MODEL_NAME}/checkpoint_{DATASET_VAR}.pth'
checkpoint = torch.load(saving_path)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

# Custom Thresholds Creation

In [44]:
from collections import defaultdict
from sklearn.metrics import f1_score

def calculate_conf_scores(valid_loader, model, mappings):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    model.to(device)
    
    conf_scores = defaultdict(list)

    with torch.no_grad():
        for mel_spec, _, file_path in valid_loader:
            mel_spec = mel_spec.to(device)

            # Estraggo la specie corretta dal path
            correct_species = file_path[0].split("/")[-2]
            outputs = model(mel_spec)
            probs = torch.sigmoid(outputs)[0].cpu().numpy()

            for i, prob in enumerate(probs):
                species_name = list(mappings.keys())[i]
                is_correct = species_name == correct_species
                conf_scores[species_name].append((prob, is_correct))

    return conf_scores

In [45]:
def compute_best_thresholds(conf_scores, num_thresholds=100, min_thresh=0.05, max_thresh=0.95):
    thresholds = {}

    for species, values in conf_scores.items():
        probs, truths = zip(*values)
        probs = np.array(probs)
        truths = np.array(truths).astype(int)

        best_thresh = 0.1
        best_f1 = 0.0

        for thresh in np.linspace(min_thresh, max_thresh, num_thresholds):
            preds = (probs >= thresh).astype(int)
            f1 = f1_score(truths, preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

        thresholds[species] = best_thresh
        print(f"📊 {species} -> {best_thresh:.3f}, F1-score: {best_f1:.3f}")

    return thresholds



In [46]:
def compute_distribution_based_thresholds(conf_scores, min_thresh=0.05, max_thresh=0.95):
    thresholds = {}

    for species, values in conf_scores.items():
        probs, truths = zip(*values)
        probs = np.array(probs)
        truths = np.array(truths).astype(int)

        correct_probs = np.array([p for p, t in zip(probs, truths) if t == 1])

        if len(correct_probs) == 0:
            thresholds[species] = 0.2  # fallback
            print(f"⚠️ Specie: {species}, nessuna predizione corretta trovata. Soglia fallback: 0.5")
            continue

        mean_conf = np.mean(probs)
        std_conf = np.std(probs)
        threshold = np.clip(mean_conf - 2 * std_conf, min_thresh, max_thresh)

        thresholds[species] = threshold
        print(f"📊 {species} -> {threshold:.3f}, (mean={mean_conf:.3f}, std={std_conf}), samples: {len(correct_probs)}")

    return thresholds


In [47]:
valid_loader = utils.get_dataloader(dataset_config, split="valid", batch_size=1)
conf_scores = calculate_conf_scores(valid_loader, model, dataset_config["mappings"])
best_thresholds = compute_best_thresholds(conf_scores)
# best_thresholds = compute_distribution_based_thresholds(conf_scores)

📊 Aeroplane -> 0.941, F1-score: 0.640
📊 Muscicapa striata_Spotted Flycatcher -> 0.905, F1-score: 0.153
📊 Periparus ater_Coal Tit -> 0.677, F1-score: 0.140
📊 Regulus regulus_Goldcrest -> 0.350, F1-score: 0.314
📊 Rain -> 0.359, F1-score: 0.833
📊 Vegetation -> 0.941, F1-score: 0.083
📊 Troglodytes troglodytes_Eurasian Wren -> 0.914, F1-score: 0.117
📊 Erithacus rubecula_European Robin -> 0.950, F1-score: 0.067
📊 Dryocopus martius_Black Woodpecker -> 0.132, F1-score: 0.900
📊 None -> 0.050, F1-score: 0.016
📊 Certhia familiaris_Eurasian Treecreeper -> 0.100, F1-score: 0.000
📊 Phylloscopus collybita_Common Chiffchaff -> 0.932, F1-score: 0.577
📊 Wind -> 0.623, F1-score: 0.538
📊 Turdus merula_Eurasian Blackbird -> 0.941, F1-score: 0.146
📊 Loxia curvirostra_Common Crossbill -> 0.050, F1-score: 0.807
📊 Regulus ignicapilla_Common Firecrest -> 0.686, F1-score: 0.524
📊 Pecking -> 0.350, F1-score: 0.432
📊 Dendrocopos major_Great Spotted Woodpecker -> 0.786, F1-score: 0.222
📊 Sylvia atricapilla_Eurasian

# Test Model

In [48]:
from sklearn.metrics import average_precision_score, precision_recall_fscore_support

def compute_samplewise_mAP(y_true, y_probs):
    """
    Calcola la sample-wise mAP (media delle AP per ogni sample).
    """
    ap_per_sample = []
    for i in range(y_true.shape[0]):
        if np.sum(y_true[i]) == 0:
            continue  # Evita sample senza label positive
        ap = average_precision_score(y_true[i], y_probs[i])
        ap_per_sample.append(ap)
    return np.mean(ap_per_sample)

def compute_classwise_mAP(y_true, y_probs):
    """
    Calcola la class-wise mAP (media delle AP per ogni classe).
    """
    ap_per_class = []
    for i in range(y_true.shape[1]):
        if np.sum(y_true[:, i]) == 0:
            continue  # Evita classi mai presenti
        ap = average_precision_score(y_true[:, i], y_probs[:, i])
        ap_per_class.append(ap)
    return np.mean(ap_per_class)

def compute_f05(y_true, y_pred):
    _, _, f05, _ = precision_recall_fscore_support(
        y_true, y_pred, beta=0.5, average='macro', zero_division=0
    )
    return f05

In [49]:
inverse_mappings = {value: key for key, value in mappings.items()}

In [50]:
import numpy as np
from sklearn.metrics import classification_report
import csv
import os

def test_model(model, dataset_config, batch_size=1, thresholds=0.5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\n🧬 Advanced testing on: {device}")
    test_pred_segments = {}

    test_loader = utils.get_dataloader(dataset_config, split="test", batch_size=batch_size, shuffle=False)
    model.eval()
    criterion = nn.BCEWithLogitsLoss()
    class_names = list(dataset_config['mappings'].keys())
    total_loss = 0.0
    all_preds = []
    all_probs = []
    all_labels = []

    use_custom_threshold = isinstance(thresholds, dict)

    with torch.no_grad():
        for mel_spec, labels, file_path in test_loader:
            basename = os.path.splitext(file_path[0].split("/")[-1])[0]
            date, time, segm1, segm2 = basename.split("_")
            audio_name = "_".join([date, time]) + ".WAV"
            segm = "_".join([segm1, segm2])
            test_pred_segments.setdefault(audio_name, {})
            test_pred_segments[audio_name].setdefault(segm, {})

            mel_spec = mel_spec.to(device)
            labels = labels.to(device)

            outputs = model(mel_spec)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            probs = torch.sigmoid(outputs)

            if use_custom_threshold:
                batch_preds = torch.zeros_like(probs)
                for i, class_name in enumerate(class_names):
                    thresh = thresholds.get(class_name, 0.5)
                    batch_preds[:, i] = (probs[:, i] > thresh).float()
            else:
                batch_preds = (probs > thresholds).float()

            correct_probs = probs * batch_preds
            conf_scores = {
                inverse_mappings[i]: correct_probs[0, i].item()
                for i in range(correct_probs.size(1))
                if correct_probs[0, i].item() != 0
            }
            test_pred_segments[audio_name][segm].update(conf_scores)

            all_probs.append(probs.cpu())
            all_preds.append(batch_preds.cpu())
            all_labels.append(labels.cpu())

    avg_loss = total_loss / len(test_loader)
    all_probs = torch.cat(all_probs).numpy()
    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    samplewise_map = compute_samplewise_mAP(all_labels, all_probs)  # chiamata mAP
    classwise_map = compute_classwise_mAP(all_labels, all_probs)    # chiamata cmAP
    f05_score = compute_f05(all_labels, all_preds)

    with open(f"models/{MODEL_NAME}/metrics_output.csv", mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Metric", "Value"])
        writer.writerow(["mAP (sample-wise)", samplewise_map])
        writer.writerow(["cmAP (class-wise)", classwise_map])
        writer.writerow(["F0.5 Score", f05_score])

    # 👇 Report
    clf_report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True, zero_division=0)
    return avg_loss, clf_report, samplewise_map, classwise_map, f05_score, test_pred_segments

In [51]:
avg_loss, clf_report, samplewise_map, classwise_map, f05_score, test_pred_segments = test_model(model, dataset_config, thresholds=best_thresholds)


🧬 Advanced testing on: cuda


In [52]:
torch.cuda.empty_cache()

In [53]:
os.makedirs(f"{MODEL_PATH}/{DATASET_VAR}", exist_ok=True)
with open(f"{MODEL_PATH}/{DATASET_VAR}/test_pred_segments.json", "w") as f:
    json.dump(test_pred_segments, f)

In [54]:
print("mAP Score: ", samplewise_map)
print("mcAP Score: ", classwise_map)
print("F0.5 Score: ", f05_score)

mAP Score:  0.6353736532286521
mcAP Score:  0.39612751276181424
F0.5 Score:  0.386404622824282


In [55]:
from io import StringIO
import pandas as pd

clf_report_df = pd.read_json(StringIO(json.dumps(clf_report)), orient='index')
clf_report_df

,precision,recall,f1-score,support
Aeroplane,0.000000,0.000000,0.000000,22
Muscicapa striata_Spotted Flycatcher,0.000000,0.000000,0.000000,14
Periparus ater_Coal Tit,0.965517,0.848485,0.903226,132
Regulus regulus_Goldcrest,0.935412,0.981308,0.957811,428
Rain,0.333333,0.347826,0.340426,23
Vegetation,0.000000,0.000000,0.000000,26
Troglodytes troglodytes_Eurasian Wren,0.604106,0.738351,0.664516,279
Erithacus rubecula_European Robin,0.500000,0.012750,0.024867,549
Dryocopus martius_Black Woodpecker,0.000000,0.000000,0.000000,21
None,0.912174,0.205043,0.334823,5116


In [56]:
with open(f"{MODEL_PATH}/classification_report_{DATASET_VAR}.json", "w") as f:
    json.dump(clf_report, f)

In [57]:
torch.cuda.empty_cache()